# A/B-тест на изменение среднего чека

У нас есть данные по транзакциям пользователей в тестовый период.  
Посмотрим воздействие эксперимента на средний чек.  
H0: средний чек в тестовой и конрольной группах не отличается.  
Уровень значимости примем 0.05.

Поскольку мы имеем дело с метрикой отношений (числитель и знаменатель не являются независимыми), то нельзя использовать базовый способ расчета дисперсии.  
Рассмотрим работу с дельта-методом и линеаризацией.

## Чтение и проверка данных

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as sps

In [6]:
df = pd.read_csv('./data_ignored/synthetic_gmv_data_1.2.csv')
df.head()

,user_id,gmv,group_name
0,myo4ixol31,1428,test
1,myo4ixol31,1428,test
2,myo4ixol31,1071,test
3,myo4ixol31,1071,test
4,pkzf2889ww,351,test


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799061 entries, 0 to 799060
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     799061 non-null  object
 1   gmv         799061 non-null  int64 
 2   group_name  799061 non-null  object
dtypes: int64(1), object(2)
memory usage: 18.3+ MB


Пропусков в данных нет.

## Преобразование данных

Посчитаем суммы покупок и их количество для каждого покупателя:

In [8]:
df_agg = df.groupby(['user_id', 'group_name']).agg(['sum', 'count'])

In [9]:
df_agg.head()

gmv      
                        sum count
user_id    group_name            
00062h7u56 control      733     1
00074uxybk test        3187     3
000ic5j18m control     2933     6
000plmykri test        1695     5
00174ganru control     1496     4

In [10]:
df_agg = df_agg.reset_index()

In [11]:
df_agg.head()

user_id group_name   gmv      
                           sum count
0  00062h7u56    control   733     1
1  00074uxybk       test  3187     3
2  000ic5j18m    control  2933     6
3  000plmykri       test  1695     5
4  00174ganru    control  1496     4

Разделим участников на тест и контроль:

In [12]:
df_test = df_agg[df_agg.group_name == 'test']
df_control = df_agg[df_agg.group_name == 'control']

In [13]:
df_agg.shape[0] == df_test.shape[0] + df_control.shape[0]

True

In [14]:
print(
        f'Количество пользователей в контроле - {df_control.shape[0]} \n'
        f'Количество пользователей в тесте    -  {df_test.shape[0]}\n'
        f'Соотношение контроля и теста - {df_control.shape[0] / df_test.shape[0]}'
)

Количество пользователей в контроле - 147291 
Количество пользователей в тесте    -  49100
Соотношение контроля и теста - 2.999816700610998


У нас нет пользователей, находящихся одновременно и в тесте, и в контроле:

In [15]:
set(df_test.user_id) & set(df_control.user_id)

set()

Подготовим вектора чисел в числителе (num) и знаменателе (denom) для теста (x) и контроля (y):

In [16]:
x_num = df_test['gmv']['sum']  # числитель теста
x_denom = df_test['gmv']['count']
y_num = df_control['gmv']['sum']
y_denom = df_control['gmv']['count']
# количество участников в тесте и контроле
n = x_num.shape[0]
m = y_num.shape[0]

## Дельта-метод

Формула дисперсии дельта-метода:
$$
\hat{\mathbf{Var}} (\dfrac{\bar X}{\bar Y} ) = \dfrac{1}{n} \dfrac{1}{\bar Y^2} \left[ \hat\sigma_X^2  - \dfrac{\bar X}{\bar Y}\hat\sigma_{XY} + \dfrac{\bar X^2}{\bar Y^2} \hat\sigma_Y^2 \right ]
$$
Статистика эксперимента будет:

$$\dfrac{\dfrac{\bar X_t}{\bar Y_t} - \dfrac{\bar X_c}{\bar Y_c}}{\sqrt{\hat{\mathbf{Var}} (\dfrac{\bar X_t}{\bar Y_t})  + \hat{\mathbf{Var}} (\dfrac{\bar X_c}{\bar Y_c}) }}$$

In [27]:
def safe_divide(x, y):
    """
    Безопасное деление: без падения при делении на ноль
    """
    try:
        return x / y
    except ZeroDivisionError:
        return np.nan

def delta_var(numerator, denominator):
    """
    Функция для расчета дисперсии дельта-методом, numerator - вектор числитель, denominator - вектор знаменатель
    """
    x = numerator
    y = denominator
    n = len(x)
    mu_x = np.mean(x)
    mu_y = np.mean(y)
    var_x = np.var(x, ddof=1)
    var_y = np.var(y, ddof=1)
    cov_xy = np.cov(x, y, ddof=1)[0][1]    
    delta_var = (
            safe_divide(
                    safe_divide(var_x, mu_y**2) 
                    - 2 * cov_xy * safe_divide(mu_x, mu_y**3) 
                    + var_y * safe_divide(mu_x**2, mu_y**4)
                    , n
            )
    )
    return delta_var


Посчитаем дельта-методом дисперсию для теста и контроля, а затем и стандартную ошибку эксперимента:

In [28]:
test_var = delta_var(x_num, x_denom)
control_var = delta_var(y_num, y_denom)
sigma = np.sqrt(test_var + control_var)
print(f'Дисперсия теста    - {test_var} \nДисперсия контроля - {control_var} \nСтандартная ошибка - {sigma}')

Дисперсия теста    - 2.1727173707916947 
Дисперсия контроля - 0.7142278796886403 
Стандартная ошибка - 1.6991013067149159


In [29]:
delta_estimator = safe_divide(np.sum(x_num), np.sum(x_denom)) - safe_divide(np.sum(y_num), np.sum(y_denom))
print(
        f'Глобальное среднее теста     - {safe_divide(np.sum(x_num), np.sum(x_denom))}\n'\
        f'Глобальное среднее контроля  - {safe_divide(np.sum(y_num), np.sum(y_denom))}\n'\
        f'Разница между этими средними -   {delta_estimator}'
)

Глобальное среднее теста     - 704.2059474502006
Глобальное среднее контроля  - 700.2234717185278
Разница между этими средними -   3.982475731672821


Посчитаем статистику t-теста и p-value для нашего эксперимента:

In [44]:
tt = safe_divide(delta_estimator, sigma)
p_value = 2 * sps.t.sf(np.abs(tt), n + m - 2)
print(f'p-value для дельта-метода - {p_value}')

p-value для дельта-метода - 0.01908571860834965


## Линеаризация

Перейдем к другой метрике L. Воспользуемся формулой для модифицированной линеаризации:
$$L(u) = \dfrac{\bar X}{\bar Y} + \dfrac{1}{\bar Y} X(u) - \dfrac{\bar X}{\bar Y^2} Y(u) = R +  \dfrac{1}{\bar Y}(X(u) - R Y(u))$$

In [ ]:
def linearization_modified(x_num, x_denom, y_num, y_denom):
    """
    Функция для расчета методом линеаризации
    """
    x_num_bar = np.mean(x_num)
    y_num_bar = np.mean(y_num)
    x_denom_bar = np.mean(x_denom)
    y_denom_bar = np.mean(y_denom)
    x_estimator = safe_divide(x_num_bar, x_denom_bar)
    y_estimator = safe_divide(y_num_bar, y_denom_bar)
    x_linear = x_estimator  + safe_divide(1, x_denom_bar)*(np.array(x_num) - x_estimator*np.array(x_denom))
    y_linear = y_estimator  + safe_divide(1, y_denom_bar)*(np.array(y_num) - y_estimator*np.array(y_denom))
    t_stat, p_value = sps.ttest_ind(x_linear, y_linear, equal_var=False)
    return t_stat, p_value

In [43]:
_, p_value = linearization_modified(x_num, x_denom, y_num, y_denom)
print(f'p-value для линеаризации - {p_value}')

p-value для линеаризации - 0.019087054199213756


p-value для обоих методов показали схожие результаты около 0.019.  
Он ниже уровня принятого уровня значимости, а значит нам надо *отвергнуть* нулевую гипотезу.